In [304]:
from keras.applications.vgg16 import VGG16, preprocess_input
import keras.utils as image
import numpy as np
import pandas as pd
import os
import keras
import random
import cv2
import faiss

random.seed(0)

## Extract the features

In [108]:
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3))
intermediate_layer_model = keras.Model(inputs=base_model.input,
                                       outputs=base_model.get_layer("fc1").output)

In [91]:
#Load images, preprocess, and extract features
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = model.predict(img_preprocessed)
    return features.flatten()
    #Extract the features and save the results to a CSV file

def process_images_and_save_features(source_dir, output_csv_file, model):
    data = {'filename': [], 'features': []}
    
    for subdir, _, files in os.walk(source_dir):
        category_name = os.path.basename(subdir)
        if files:
            print(f"Processing category: {category_name}")
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(subdir, file)
                features = extract_features(img_path, model)
                save_img_path = os.path.join('./images', category_name, file)
                data['filename'].append(save_img_path)
                data['features'].append(features)
    

    df = pd.DataFrame(data)
    df['features'] = df['features'].apply(lambda x: ','.join(x.astype(str)))  # Converts an array of features to a comma-separated string
    df.to_csv(output_csv_file, index=False)
    print("Features are successfully saved to CSV.")


source_directory = './images'
output_csv_file = 'features_database.csv'
process_images_and_save_features(source_directory, output_csv_file, intermediate_layer_model)


Processing category: after-images
Processing category: 005.Crested_Auklet
1/1 [==============================] - 0s 137ms/step
Processing category: 015.Lazuli_Bunting
1/1 [==============================] - 0s 128ms/step
Processing category: 156.White_eyed_Vireo
1/1 [==============================] - 0s 131ms/step
Processing category: 081.Pied_Kingfisher
1/1 [==============================] - 0s 141ms/step
Processing category: 135.Bank_Swallow
1/1 [==============================] - 0s 133ms/step
Processing category: 200.Common_Yellowthroat
1/1 [==============================] - 0s 130ms/step
Processing category: 086.Pacific_Loon
1/1 [==============================] - 0s 159ms/step
Processing category: 067.Anna_Hummingbird
1/1 [==============================] - 0s 143ms/step
Processing category: 076.Dark_eyed_Junco
1/1 [==============================] - 0s 146ms/step
Processing category: 149.Brown_Thrasher
1/1 [==============================] - 0s 158ms/step
Processing category: 127.Sava

In [ ]:
df = pd.read_csv('features_database.csv')

print(df.head(10))

# Analyze statistical data for features, such as maximum, minimum, and average values
feature_stats = df['features'].apply(lambda x: pd.Series([float(i) for i in x.split(',')]))
print(feature_stats.describe())

## Apply Approximate Nearest Neighbor Function

In [166]:
df = pd.read_csv('features_database.csv')

In [167]:
print(df['features'])

0        5.522972,0.46964902,0.0,1.56195,2.6486506,1.76...
1        0.0,0.0,0.0,0.0,0.0,0.0,2.2239554,0.0,0.0,0.0,...
2        0.0,0.0,0.0,0.0,0.11435537,0.0,31.162338,1.168...
3        0.0,0.37464738,3.544683,0.0,4.0713625,0.0,5.18...
4        0.0,3.8946447,0.0,4.5498357,0.0,0.0,2.8929832,...
                               ...                        
11783    6.893283,3.2797723,0.0,8.032077,0.0,0.0,14.800...
11784    1.2642565,0.0,0.0,0.0,0.0,0.0,3.2267692,0.0,2....
11785    6.0339174,9.147993,0.0,0.0,0.0,0.6833972,8.336...
11786    3.0676713,7.8004546,0.0,32.109627,0.0,0.0,1.14...
11787    4.465256,0.0,0.0,7.747105,0.0,0.0,0.0,0.0,9.14...
Name: features, Length: 11788, dtype: object


In [168]:


features_df = df['features'].to_numpy()
features_all = []
for feat in features_df:
    feat = feat.split(',')
    feat = [float(f) for f in feat]
    features_all.append(feat)
features_all = np.array(features_all)
print(features_all[:5])


[[5.522972   0.46964902 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.37464738 3.544683   ... 0.         0.         0.        ]
 [0.         3.8946447  0.         ... 0.         0.         0.        ]]


Apply SIFT

In [178]:
def compute_ann(features_all, features_cnn_q):
    dimension = features_all.shape[1]
    index = faiss.IndexFlatL2(dimension)  # Using L2 distance for similarity
    index.add(features_all)  # Add the dataset to the index

    # Perform the search
    k = 50  # Number of nearest neighbors to find
    D, I = index.search(features_cnn_q, k)  # D is the distance, I is the indices of the nearest neighbors

In [283]:
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = model.predict(img_preprocessed)
    return features.flatten()

def get_query_cnn_features(root):
    files = os.listdir(root)
    features_q = []
    for f in files:
        path = os.path.join(root, f)
        if path.endswith(('.png', '.jpg')):
            feat = extract_features(path, intermediate_layer_model)
            features_q.append(feat)

    features_q = np.array(features_q)
    return features_q

def get_query_sift_features(root):
    files = os.listdir(root)
    sift = cv2.SIFT.create()
    sift_features = []
    for f in files:
        path = os.path.join(root, f)
        if path.endswith(('.jpg', '.png', '.jpeg')):
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        else: continue
        _, descriptors = sift.detectAndCompute(img, None)
        sift_features.append(descriptors)
    features_sift_q = np.concatenate(sift_features)
    return features_sift_q

def compute_ann(features_all,root):
    features_cnn_q = get_query_cnn_features(root)
    dimension = features_all.shape[1]
    index = faiss.IndexFlatL2(dimension)  # Using L2 distance for similarity
    index.add(features_all)  # Add the dataset to the index

    # Perform the search
    k = 50  # Number of nearest neighbors to find
    D, I = index.search(features_cnn_q, k)
    return I, D

def compute_score(features_all, root, cnn_ratio, sift_ratio):
    Index, Distance = compute_ann(features_all,root)
    features_sift_q = get_query_sift_features(root)

    scoring_dict = {}
    indices = Index.flatten()
    distances = Distance.flatten()
    
    sift = cv2.SIFT.create()
    
    for key, value in zip(indices, distances):
        cnn_score = 1/value
        # print(cnn_score)
        f = df.iat[key, 0]

        img = cv2.imread(f,cv2.IMREAD_GRAYSCALE)
        _, descriptors = sift.detectAndCompute(img, None)
        index_params = dict(algorithm = 1, trees = 5)
        search_params = dict()
        # Create the FLANN matcher
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(descriptors, features_sift_q, k=2)
        good_matches = 0
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches+=1

        sift_score = good_matches/min(len(descriptors), len(features_sift_q))
        # print(sift_score)

        score = cnn_ratio*cnn_score + sift_ratio*sift_score

        if key in scoring_dict:
            scoring_dict[key] += cnn_ratio*cnn_score
        else:
            scoring_dict[key] = score
    

    return scoring_dict

In [294]:
root = "./test_img/trial3"
cnn_ratio = 1000
sift_ratio = 0.2
scoring_dict = compute_score(features_all, root, cnn_ratio, sift_ratio)
top_three_indecies = [key for key, value in sorted(scoring_dict.items(), key=lambda item: item[1], reverse=True)[:3]]

1/1 [==============================] - 0s 139ms/step


In [295]:
for i in top_three_indecies:

    print(df.iloc[i,0])

./images/063.Ivory_Gull/Ivory_Gull_0101_49790.jpg
./images/128.Seaside_Sparrow/Seaside_Sparrow_0026_120798.jpg
./images/028.Brown_Creeper/Brown_Creeper_0111_24590.jpg
